In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.mida import Mida
from models.mice import Mice

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from fancyimpute import KNN, SoftImpute

from preprocessing.data_loading import DataSet
from preprocessing.missing_value_generation import mcar_generator, mar_generator, mnar_generator
from models.imputation_wrapper import SingleImputationWrapper, MultiImputationWrapper
from analysis.evaluation import OutlierEvaluation, LnormEvaluation, TimeEvaluation
from analysis.method_wrapper import OutlierMethodWrapper

/home/marcus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
datasets_df = pd.read_csv('data/datasets.csv')
datasets = datasets_df.loc[datasets_df['type']=='outlier', 'dataset'].tolist()
print(datasets)

['ctg', 'wine', 'pima', 'annthyroid', 'vowels', 'pendigits', 'optdigits', 'musk']


In [4]:
amputations = [mcar_generator(), mar_generator(2), mnar_generator(2)]

In [5]:
p_range = [0.2, 0.4, 0.6, 0.8]

In [6]:
# each dataset
for dataset_name in datasets:    
    print(dataset_name + ' started-------------------------------------------')
    # init dataset
    dataset = DataSet(dataset_name, 'outlier')

    # init evaluation
    out_evaluation = OutlierEvaluation(OutlierMethodWrapper(IsolationForest()), dataset)
    lnorm_evaluation = LnormEvaluation(dataset)
    time_evaluation = TimeEvaluation(dataset)

    # result on complete dataset
    out_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')
    lnorm_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')

    # define methods for imputation
    imputations = [SingleImputationWrapper(KNN(3, verbose=False)), SingleImputationWrapper(SoftImpute(verbose=False)),
                   MultiImputationWrapper(Mice(40, verbose=False)), MultiImputationWrapper(Mida(40, verbose=False))]

    # different methods
    for amputation in amputations:
        dataset.init_missing_data(amputation)

        # increasing rate p
        for p in p_range:
            dataset.ampute_values(p)

            # apply each imputation
            for imputation in imputations:
                results, exec_time = imputation.complete(dataset.missing_data())
                out_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
                lnorm_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
                time_evaluation.evaluate_result(exec_time, p, amputation.name(), imputation.name(), imputation.number())
            print(amputation.name() + ': ' + str(p) + ' done')
    
    # dump results
    out_evaluation.dump_results()
    lnorm_evaluation.dump_results()
    time_evaluation.dump_results()
    print(dataset_name + ' done-------------------------------------------')

ctg started-------------------------------------------


/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan
/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


mcar: 0.2 done


/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result[c][rows] = np.nan
/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


mcar: 0.4 done


...

mcar: 0.2 done


/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


mcar: 0.4 done


/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


KeyboardInterrupt: 